# Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama

#############################################
### İş Problemi

Gezinomi yaptığı satışların bazı özelliklerini kullanarak seviye tabanlı (level based) yeni satış tanımları oluşturmak ve bu yeni satış tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.


Örneğin: Antalya’dan Herşey Dahil bir otele yoğun bir dönemde gitmek isteyen bir müşterinin ortalama ne kadar kazandırabileceği belirlenmek isteniyor.

#############################################

In [35]:
# Kütüphanemizi import edelim;

import pandas as pd

# Pandas ayarlamalarımızı yapaılım;

pd.set_option("display.max_rows", None)                       # Bütün satırları görmek istersek bu kodu çalıştırırız.
pd.set_option("display.max_columns", None)                      # Bütün sütunları görmek için 

In [36]:
# Verimizi okutalım;

df = pd.read_excel('datasets/miuul_gezinomi.xlsx')

# Virgülden sonra iki basamak görmek için 
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Verimize hızlı bir bakış atalım;
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.30,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.97,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.84,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.71,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.48,Yarım Pansiyon,İzmir,Saturday,2,Low


In [37]:
# Verimizin boyutuna bakalım;

df.shape

(59164, 9)

In [38]:
# Değişkenlerimize bakalım;

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59164 entries, 0 to 59163
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SaleId              59164 non-null  int64         
 1   SaleDate            59164 non-null  datetime64[ns]
 2   CheckInDate         59164 non-null  datetime64[ns]
 3   Price               59151 non-null  float64       
 4   ConceptName         59164 non-null  object        
 5   SaleCityName        59164 non-null  object        
 6   CInDay              59164 non-null  object        
 7   SaleCheckInDayDiff  59164 non-null  int64         
 8   Seasons             59164 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 4.1+ MB


    Yukarıda görülüyor ki 8 farklı değişkenimiz var ve boş değerimiz yok.

In [ ]:
# SaleId                : Satış id
# SaleDate              : Satış Tarihi
# Price                 : Satış için ödenen fiyat
# ConceptName           : Otel konsept bilgisi
# SaleCityName          : Otelin bulunduğu şehir bilgisi
# CheckInDate           : Müşterinin otelegirişitarihi
# CInDay                : Müşterinin otele giriş günü
# SaleCheckInDayDiff    : Check in ile giriş tarihi gün farkı
# Season                : Otele giriş tarihindeki sezon bilgisi

In [39]:
# Veri setinde kaç farklı şehir olduğuna bakalım;
print(df["SaleCityName"].nunique())

print("~~~~~~~~~~~~~~~~~")
# Bu şehirlerin frekansına bakalım;
print(df["SaleCityName"].value_counts())

6
~~~~~~~~~~~~~~~~~
Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: SaleCityName, dtype: int64


    6 farklı şehir var ve insanlar en çok antalyaya gitmeyi tercih ederken, en az gitmeyi tercih ettikleri yer ise Girneymiş.

In [40]:
# Kaç unique Concept olduğuna bakalım;
print(df["ConceptName"].nunique())

print("~~~~~~~~~~~~~~~~~~~~~~~~~~")
# Hangi Concept'dan kaçar tane satış gerçekleştiğine bakalım;
print(df["ConceptName"].value_counts())

3
~~~~~~~~~~~~~~~~~~~~~~~~~~
Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: ConceptName, dtype: int64


    3 farklı konsept olup, müşteriler genel olarak "Herşey Dahil" tatili tercih ediyormuş.

In [41]:
# Toplam satışlara şehir kırılımında bakalım;
df.groupby("SaleCityName").agg({"Price": "sum"}).sort_values(by="Price", ascending=False)

,Price
SaleCityName,
Antalya,2041911.10
Muğla,665842.21
Aydın,573296.01
İzmir,165934.83
Diğer,154572.29
Girne,27065.03


    En çok para kazanılan il Antalya iken, en az para kazanılan lokasyonun ise Girne olduğunu görüyoruz. Zaten doğal olarak insanların en çok Antalya'yı tercih etmesinden de bunu kestirebilirdik.

In [42]:
# Toplam satışlara konsept kırılımında bakalım;
df.groupby("ConceptName").agg({"Price": "sum"}).sort_values(by="Price", ascending=False)

,Price
ConceptName,
Herşey Dahil,3332910.77
Yarım Pansiyon,174402.35
Oda + Kahvaltı,121308.35


In [43]:
# Ortalama tatil fiyatına şehir kırılımında bakalım;
df.groupby(by=['SaleCityName']).agg({"Price": "mean"}).sort_values(by="Price", ascending=False)

,Price
SaleCityName,
İzmir,66.27
Antalya,64.52
Muğla,62.46
Girne,59.48
Aydın,53.86
Diğer,47.71


    En pahalı lokasyon İzmir olarak gözükmektedir.

In [44]:
# Ortalama tatil fiyatına concept kırılımında bakalım;
df.groupby(by=['ConceptName']).agg({"Price": "mean"}).sort_values(by="Price", ascending=False)

,Price
ConceptName,
Herşey Dahil,62.67
Oda + Kahvaltı,50.25
Yarım Pansiyon,49.03


    Yukarıda ortalama olarak bakılıdığında "Oda + Kahvaltı"nın "Yarım Pansiyon"dan daha pahalı olduğu gözükmektedir.

In [45]:
#  Şehir-Concept kırılımında PRICE ortalamaları nedir?
df.groupby(by=["SaleCityName", 'ConceptName']).agg({"Price": "mean"}).  \
    sort_values(by=["SaleCityName", "ConceptName"], ascending=True)

Price
SaleCityName ConceptName          
Antalya      Herşey Dahil    64.52
             Oda + Kahvaltı  63.50
             Yarım Pansiyon  67.19
Aydın        Herşey Dahil    54.00
             Oda + Kahvaltı  34.46
             Yarım Pansiyon  30.02
Diğer        Herşey Dahil    84.77
             Oda + Kahvaltı  37.60
             Yarım Pansiyon  42.11
Girne        Herşey Dahil    97.68
             Oda + Kahvaltı  39.78
             Yarım Pansiyon  53.25
Muğla        Herşey Dahil    63.02
             Oda + Kahvaltı  59.04
             Yarım Pansiyon  45.12
İzmir        Herşey Dahil    74.70
             Oda + Kahvaltı  41.32
             Yarım Pansiyon  59.61

    Girne'de "Tam Pansiyon" tatilin eksra pahalı olduğu gözüküyor.
    Aydın'da da "Yarım Pansiyon" tatilin uygun olduğunu söyleyebiliriz.

In [46]:
# Değişkenlerimizden biri de "SaleCheckInDayDiff" değişkeni, 
# yani satış yapıldıktan kaç gün sonra check in yapıldığını ifade eden bir değişken.

# Bu değişkeni kategorik bir değişken haline getirip isimlendireceğiz;

# Aralıklarımızı tanımlayalım;
bins = [-1, 7, 30, 90, df["SaleCheckInDayDiff"].max()]  
# Eğer ki bin'i 0'dan başlatırsak "SaleCheckInDayDiff" değişkeni 0 olan gözlemler için sınıflandırma yapılamaz.
# Dilerseniz 0 yapıp değişimi gözlemleyebilirsiniz.

# Atayacağımız isimleri tanımlayalım;
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]

# Şimdi kategorik değişken dönüşümünü yapalım;

df["EB_Score"] = pd.cut(df["SaleCheckInDayDiff"], bins, labels=labels)
df.head(10)

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons,EB_Score
0,415122,2022-12-03,2022-12-03,79.30,Herşey Dahil,Antalya,Saturday,0,Low,Last Minuters
1,415103,2022-12-03,2022-12-03,45.97,Yarım Pansiyon,Antalya,Saturday,0,Low,Last Minuters
2,404034,2022-09-12,2022-09-13,77.84,Herşey Dahil,Antalya,Tuesday,1,High,Last Minuters
3,415094,2022-12-03,2022-12-10,222.71,Yarım Pansiyon,İzmir,Saturday,7,Low,Last Minuters
4,414951,2022-12-01,2022-12-03,140.48,Yarım Pansiyon,İzmir,Saturday,2,Low,Last Minuters
5,415091,2022-12-03,2022-12-03,70.27,Yarım Pansiyon,İzmir,Saturday,0,Low,Last Minuters
6,415085,2022-12-03,2022-12-09,45.79,Yarım Pansiyon,Antalya,Friday,6,Low,Last Minuters
7,415084,2022-12-03,2022-12-03,51.24,Herşey Dahil,Antalya,Saturday,0,Low,Last Minuters
8,415081,2022-12-03,2022-12-04,77.29,Yarım Pansiyon,İzmir,Sunday,1,Low,Last Minuters
9,415079,2022-12-03,2022-12-03,68.68,Yarım Pansiyon,Diğer,Saturday,0,Low,Last Minuters


In [48]:
df.groupby(by=["EB_Score" ]).agg({"Price": ["mean", "count"]})

Price       
                    mean  count
EB_Score                       
Last Minuters      59.57  26558
Potential Planners 61.30  16617
Planners           64.77   9182
Early Bookers      63.76   6794

    Müşteriler genel olarak "Last Minuters" olmakla birlikte, fiaytlar arasındaki marj (kişisel bir yorum) ççok da fazla olmadığı gözükmektedir.

In [47]:
# Şehir-Concept-EB Score kırılımında ücret ortalamalarına bakalım;
df.groupby(by=["SaleCityName", 'ConceptName', "EB_Score" ]).agg({"Price": ["mean", "count"]})

Price       
                                                 mean  count
SaleCityName ConceptName    EB_Score                        
Antalya      Herşey Dahil   Last Minuters       62.75  14148
                            Potential Planners  64.90   8874
                            Planners            67.88   4490
                            Early Bookers       66.49   3281
             Oda + Kahvaltı Last Minuters       65.35    503
                            Potential Planners  57.74     75
                            Planners            39.85     15
                            Early Bookers       35.00      5
             Yarım Pansiyon Last Minuters       70.43    204
                            Potential Planners  55.64     39
                            Planners            58.03      9
                            Early Bookers       35.03      4
Aydın        Herşey Dahil   Last Minuters       51.71   3965
                            Potential Planners  52.09   2717
                            Planners            57.88   1834
                            Early Bookers       57.44   2060
             Oda + Kahvaltı Last Minuters       37.80     20
                            Potential Planners  31.08     15
                            Planners            30.20      2
                            Early Bookers       26.81      1
             Yarım Pansiyon Last Minuters       30.87     18
                            Potential Planners  23.73      6
                            Planners            28.81      6
                            Early Bookers       59.60      1
Diğer        Herşey Dahil   Last Minuters       75.47    155
                            Potential Planners  85.79    129
                            Planners            87.26    120
                            Early Bookers       94.79    101
             Oda + Kahvaltı Last Minuters       38.11    539
                            Potential Planners  34.98    176
                            Planners            43.44     34
                            Early Bookers       35.72      8
             Yarım Pansiyon Last Minuters       39.47   1173
                            Potential Planners  45.70    605
                            Planners            46.14    173
                            Early Bookers       50.68     27
Girne        Herşey Dahil   Last Minuters       93.25     24
                            Potential Planners  94.95     41
                            Planners           103.41     25
                            Early Bookers      102.38     16
             Oda + Kahvaltı Last Minuters       37.58     62
                            Potential Planners  39.77     39
                            Planners            43.15     28
                            Early Bookers       43.97     10
             Yarım Pansiyon Last Minuters       60.86     73
                            Potential Planners  53.40     81
                            Planners            45.99     43
                            Early Bookers       33.60     13
Muğla        Herşey Dahil   Last Minuters       63.69   3851
                            Potential Planners  62.41   2942
                            Planners            63.44   1959
                            Early Bookers       61.52   1067
             Oda + Kahvaltı Last Minuters       51.09    331
                            Potential Planners  63.11    149
                            Planners            72.39    121
                            Early Bookers       66.76     53
             Yarım Pansiyon Last Minuters       41.88    104
                            Potential Planners  45.29     50
                            Planners            52.80     24
                            Early Bookers       61.14      9
İzmir        Herşey Dahil   Last Minuters       73.25    589
                            Potential Planners  67.77    440
                            Planners            80.04    231
                      

In [49]:
# Şehir-Concept-Sezon kırılımında ücret ortalamalarına bakalım;
df.groupby(by=["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": ["mean", "count"]})

Price       
                                      mean  count
SaleCityName ConceptName    Seasons              
Antalya      Herşey Dahil   High     64.92  27126
                            Low      61.55   3667
             Oda + Kahvaltı High     66.27    303
                            Low      60.67    295
             Yarım Pansiyon High     73.26    118
                            Low      62.00    138
Aydın        Herşey Dahil   High     54.95  10103
                            Low      33.68    473
             Oda + Kahvaltı High     30.39     27
                            Low      44.45     11
             Yarım Pansiyon High     32.63     20
                            Low      25.27     11
Diğer        Herşey Dahil   High     83.79    364
                            Low      87.31    141
             Oda + Kahvaltı High     34.84    349
                            Low      39.96    408
             Yarım Pansiyon High     39.73    516
                            Low      42.95   1462
Girne        Herşey Dahil   High    103.94     55
                            Low      90.94     51
             Oda + Kahvaltı High     39.48     66
                            Low      40.04     73
             Yarım Pansiyon High     58.38    100
                            Low      48.58    110
Muğla        Herşey Dahil   High     63.38   9582
                            Low      48.30    237
             Oda + Kahvaltı High     61.26    592
                            Low      37.77     62
             Yarım Pansiyon High     45.82    177
                            Low      32.68     10
İzmir        Herşey Dahil   High     74.75   1234
                            Low      74.31    147
             Oda + Kahvaltı High     54.14     86
                            Low      33.56    142
             Yarım Pansiyon High     87.66    118
                            Low      55.35    777

In [50]:
# Şehir-Concept-CInday kırılımında ücret ortalamalarına bakalım;
df.groupby(by=["SaleCityName", "ConceptName", "CInDay"]).agg({"Price": ["mean", "count"]})

Price      
                                        mean count
SaleCityName ConceptName    CInDay                
Antalya      Herşey Dahil   Friday     62.66  4136
                            Monday     63.26  6831
                            Saturday   64.42  4741
                            Sunday     65.85  3818
                            Thursday   62.89  3898
                            Tuesday    66.77  3760
                            Wednesday  67.17  3609
             Oda + Kahvaltı Friday     63.13   114
                            Monday     57.17    66
                            Saturday   58.01   117
                            Sunday     63.47    46
                            Thursday   64.65    93
                            Tuesday    65.84    73
                            Wednesday  72.81    89
             Yarım Pansiyon Friday     69.26    38
                            Monday     76.95    20
                            Saturday   63.38    71
                            Sunday     64.45    30
                            Thursday   66.64    40
                            Tuesday    71.02    21
                            Wednesday  67.78    36
Aydın        Herşey Dahil   Friday     51.03  1825
                            Monday     54.38  2566
                            Saturday   56.89  1702
                            Sunday     53.19  1098
                            Thursday   54.51  1315
                            Tuesday    53.51  1098
                            Wednesday  54.22   972
             Oda + Kahvaltı Friday     31.17     7
                            Monday     45.40     7
                            Saturday   23.93     2
                            Sunday     43.53     3
                            Thursday   37.52     7
                            Tuesday    32.93     3
                            Wednesday  25.95     9
             Yarım Pansiyon Friday     24.52     5
                            Monday     43.43     3
                            Saturday   29.70     8
                            Sunday     30.28     3
                            Thursday   22.78     5
                            Tuesday    32.16     4
                            Wednesday  35.55     3
Diğer        Herşey Dahil   Friday     84.66    78
                            Monday     90.60   112
                            Saturday   77.23    90
                            Sunday     81.61    57
                            Thursday   76.11    54
                            Tuesday    90.33    56
                            Wednesday  91.18    58
             Oda + Kahvaltı Friday     38.47   116
                            Monday     36.65    87
                            Saturday   36.40   207
                            Sunday     38.06    67
                            Thursday   35.39    99
                            Tuesday    42.24    80
                            Wednesday  38.05   101
             Yarım Pansiyon Friday     42.37   407
                            Monday     40.28   217
                            Saturday   43.25   638
                            Sunday     41.86   219
                            Thursday   41.95   198
                            Tuesday    43.50   140
                            Wednesday  38.74   159
Girne        Herşey Dahil   Friday    121.19    12
                            Monday     98.66    22
                            Saturday  109.17     9
                            Sunday     85.19    38
                            Thursday  112.38     7
                            Tuesday   111.40     8
                            Wednesday  83.17    10
             Oda + Kahvaltı Friday     45.02    34
                            Monday     40.76    13
                            Saturday   37.18    29
                            Sunday     38.22    20
                            Thursday   36.50    13
                            Tuesday    35.39    19
          

In [51]:
# İki önceki kısımdaki çıktıyı çıktıyı daha iyi görebilmek için price'ı azalan bir şekilde sıralayalım;

agg_df = df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": "mean"}). \
    sort_values("Price", ascending=False)

agg_df.head(20)


Price
SaleCityName ConceptName    Seasons       
Girne        Herşey Dahil   High    103.94
                            Low      90.94
İzmir        Yarım Pansiyon High     87.66
Diğer        Herşey Dahil   Low      87.31
                            High     83.79
İzmir        Herşey Dahil   High     74.75
                            Low      74.31
Antalya      Yarım Pansiyon High     73.26
             Oda + Kahvaltı High     66.27
             Herşey Dahil   High     64.92
Muğla        Herşey Dahil   High     63.38
Antalya      Yarım Pansiyon Low      62.00
             Herşey Dahil   Low      61.55
Muğla        Oda + Kahvaltı High     61.26
Antalya      Oda + Kahvaltı Low      60.67
Girne        Yarım Pansiyon High     58.38
İzmir        Yarım Pansiyon Low      55.35
Aydın        Herşey Dahil   High     54.95
İzmir        Oda + Kahvaltı High     54.14
Girne        Yarım Pansiyon Low      48.58

In [52]:
# agg_df'deki indeksleri değişkene çevirelim;

agg_df.reset_index(inplace=True)
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.94
1,Girne,Herşey Dahil,Low,90.94
2,İzmir,Yarım Pansiyon,High,87.66
3,Diğer,Herşey Dahil,Low,87.31
4,Diğer,Herşey Dahil,High,83.79


In [55]:
# Yeni level based satışları tanımlayalım ve veri setine değişken olarak ekleyelim;

agg_df['sales_level_based'] = agg_df[["SaleCityName", "ConceptName", "Seasons"]].   \
    agg(lambda x: '_'.join(x).upper(), axis=1)
agg_df.head(10)

,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,Girne,Herşey Dahil,High,103.94,GIRNE_HERŞEY DAHIL_HIGH
1,Girne,Herşey Dahil,Low,90.94,GIRNE_HERŞEY DAHIL_LOW
2,İzmir,Yarım Pansiyon,High,87.66,İZMIR_YARIM PANSIYON_HIGH
3,Diğer,Herşey Dahil,Low,87.31,DIĞER_HERŞEY DAHIL_LOW
4,Diğer,Herşey Dahil,High,83.79,DIĞER_HERŞEY DAHIL_HIGH
5,İzmir,Herşey Dahil,High,74.75,İZMIR_HERŞEY DAHIL_HIGH
6,İzmir,Herşey Dahil,Low,74.31,İZMIR_HERŞEY DAHIL_LOW
7,Antalya,Yarım Pansiyon,High,73.26,ANTALYA_YARIM PANSIYON_HIGH
8,Antalya,Oda + Kahvaltı,High,66.27,ANTALYA_ODA + KAHVALTI_HIGH
9,Antalya,Herşey Dahil,High,64.92,ANTALYA_HERŞEY DAHIL_HIGH


In [56]:
#  Personları PRICE'a göre segmentlere ayıralım;

agg_df["SEGMENT"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])
agg_df

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
0,Girne,Herşey Dahil,High,103.94,GIRNE_HERŞEY DAHIL_HIGH,A
1,Girne,Herşey Dahil,Low,90.94,GIRNE_HERŞEY DAHIL_LOW,A
2,İzmir,Yarım Pansiyon,High,87.66,İZMIR_YARIM PANSIYON_HIGH,A
3,Diğer,Herşey Dahil,Low,87.31,DIĞER_HERŞEY DAHIL_LOW,A
4,Diğer,Herşey Dahil,High,83.79,DIĞER_HERŞEY DAHIL_HIGH,A
5,İzmir,Herşey Dahil,High,74.75,İZMIR_HERŞEY DAHIL_HIGH,A
6,İzmir,Herşey Dahil,Low,74.31,İZMIR_HERŞEY DAHIL_LOW,A
7,Antalya,Yarım Pansiyon,High,73.26,ANTALYA_YARIM PANSIYON_HIGH,A
8,Antalya,Oda + Kahvaltı,High,66.27,ANTALYA_ODA + KAHVALTI_HIGH,A
9,Antalya,Herşey Dahil,High,64.92,ANTALYA_HERŞEY DAHIL_HIGH,B


In [66]:
# Segment kırılımında fiyatları inceleyelim;
agg_df.groupby("SEGMENT").agg({"Price": ["mean", "max", "sum"]}).sort_values(by="SEGMENT", ascending=False)

Price              
         mean    max    sum
SEGMENT                    
A       82.47 103.94 742.21
B       60.27  64.92 542.47
C       44.89  54.14 403.99
D       33.37  39.48 300.30

In [68]:
# agg_df'yi price'a göre sıralayalım;
agg_df.sort_values(by="Price", ascending=False)

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
0,Girne,Herşey Dahil,High,103.94,GIRNE_HERŞEY DAHIL_HIGH,A
1,Girne,Herşey Dahil,Low,90.94,GIRNE_HERŞEY DAHIL_LOW,A
2,İzmir,Yarım Pansiyon,High,87.66,İZMIR_YARIM PANSIYON_HIGH,A
3,Diğer,Herşey Dahil,Low,87.31,DIĞER_HERŞEY DAHIL_LOW,A
4,Diğer,Herşey Dahil,High,83.79,DIĞER_HERŞEY DAHIL_HIGH,A
5,İzmir,Herşey Dahil,High,74.75,İZMIR_HERŞEY DAHIL_HIGH,A
6,İzmir,Herşey Dahil,Low,74.31,İZMIR_HERŞEY DAHIL_LOW,A
7,Antalya,Yarım Pansiyon,High,73.26,ANTALYA_YARIM PANSIYON_HIGH,A
8,Antalya,Oda + Kahvaltı,High,66.27,ANTALYA_ODA + KAHVALTI_HIGH,A
9,Antalya,Herşey Dahil,High,64.92,ANTALYA_HERŞEY DAHIL_HIGH,B


In [69]:
# "ANTALYA_HERŞEY DAHIL_HIGH" hangi segmenttedir ve ne kadar ücret beklenmektedir?

new_user = "ANTALYA_HERŞEY DAHIL_HIGH"
agg_df[agg_df["sales_level_based"] == new_user]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
9,Antalya,Herşey Dahil,High,64.92,ANTALYA_HERŞEY DAHIL_HIGH,B
